In [5]:
# grover.py
# Grover's Algorithm for Qiskit 1.4.4
# Run with: python grover.py

# Install: pip install qiskit==1.4.4 qiskit-aer matplotlib

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import sys

def create_grover_circuit(n_qubits=2, marked_state='11'):
    """Create Grover circuit for N=4, marking |11>"""
    try:
        print(f"Building circuit for {n_qubits} index qubits, marking |{marked_state}>")
        qc = QuantumCircuit(n_qubits + 1, n_qubits)

        # 1. Superposition
        print("  Applying Hadamard gates for superposition...")
        for q in range(n_qubits):
            qc.h(q)
        qc.x(n_qubits)
        qc.h(n_qubits)  # Workspace qubit to |->

        # 2. Oracle: mark |11> with CZ
        print("  Applying oracle...")
        qc.cz(0, 1)

        # 3. Diffusion
        print("  Applying diffusion operator...")
        for q in range(n_qubits):
            qc.h(q)
        for q in range(n_qubits):
            qc.x(q)
        qc.cz(0, 1)
        for q in range(n_qubits):
            qc.x(q)
        for q in range(n_qubits):
            qc.h(q)

        # 4. Measure
        print("  Adding measurements...")
        qc.measure(range(n_qubits), range(n_qubits))

        return qc
    except Exception as e:
        print(f"Error building circuit: {e}")
        sys.exit(1)

def run_simulation(qc, shots=1024):
    """Run simulation and return counts"""
    try:
        print(f"\nSimulating with {shots} shots...")
        simulator = AerSimulator()
        job = simulator.run(qc, shots=shots)
        result = job.result()
        counts = result.get_counts()
        return counts
    except Exception as e:
        print(f"Simulation error: {e}")
        sys.exit(1)

def main():
    print("GROVER'S ALGORITHM (Qiskit 1.4.4)")
    print("=" * 40)

    # Build circuit
    qc = create_grover_circuit(n_qubits=2, marked_state='11')

    # Print circuit
    print("\nCircuit Diagram:")
    print(qc.draw('text', fold=-1))

    # Simulate
    counts = run_simulation(qc, shots=1024)

    # Print results
    print("\nResults:")
    total_shots = sum(counts.values())
    for state, count in sorted(counts.items()):
        prob = count / total_shots * 100
        print(f"  |{state}> : {count}/{total_shots} ({prob:.1f}%)")

    # Plot
    print("\nGenerating histogram...")
    try:
        plot_histogram(counts)
        plt.show()
    except Exception as e:
        print(f"Plotting error: {e}")

    # Success check
    success_prob = counts.get('11', 0) / total_shots * 100
    print(f"\nResult: |11> found with {success_prob:.1f}% probability")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Main error: {e}")
        sys.exit(1)

GROVER'S ALGORITHM (Qiskit 1.4.4)
Building circuit for 2 index qubits, marking |11>
  Applying Hadamard gates for superposition...
  Applying oracle...
  Applying diffusion operator...
  Adding measurements...

Circuit Diagram:
     ┌───┐     ┌───┐┌───┐   ┌───┐┌───┐┌─┐   
q_0: ┤ H ├──■──┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├───
     ├───┤  │  ├───┤├───┤ │ ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├──■──┤ H ├┤ X ├─■─┤ X ├┤ H ├─╫─┤M├
     ├───┤┌───┐└───┘└───┘   └───┘└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├────────────────────────╫──╫─
     └───┘└───┘                        ║  ║ 
c: 2/══════════════════════════════════╩══╩═
                                       0  1 

Simulating with 1024 shots...

Results:
  |11> : 1024/1024 (100.0%)

Generating histogram...

Result: |11> found with 100.0% probability
